In [2]:
import numpy as np
from scipy.sparse.linalg import eigsh
import scipy.sparse as sparse
from scipy.linalg import eigh
import scipy.linalg
import matplotlib.pyplot as plt
from qutip import *
import math

In [3]:
#plt.rcParams['figure.figsize']=8.5,5.1
plt.rcParams['font.size']=20
plt.rcParams['lines.linewidth']=1.5
plt.rcParams['text.usetex']=True

In [4]:
#Construction of necessary elements of theta and phi Hilbert spaces

# For phi

N=50
a_phi=np.diag(np.sqrt(np.arange(1, N+1)), 1)
ad_phi=np.diag(np.sqrt(np.arange(1, N+1)), -1)
N_phi=ad_phi@a_phi
Id_N=np.eye(N+1)

# For theta

M=20
Id_M=np.eye(2*M+1)
n_theta=np.flipud(np.fliplr(np.diag(np.arange(-M,M+1), 0)))

#Values of equilibrium

phi_ext_val=np.pi
ng_val=0.5

#Qubit parameters 1 

E_Cs=1.0
E_Cj=1.0
E_L=1.0
E_J=1.0
alpha=(1j*(E_Cj/E_L)**(1.0/4.0))

# Qubit parameters 2

E_Cs_2=(1.0/20)
E_Cj_2=1.0
E_L_2=(1.0/16)
E_J_2=(1.0/5.0)
alpha2=(1j*(E_Cj_2/E_L_2)**(1.0/4.0))

#FOR EXPECTATION VALUES

#PHI
print("PHI")
print((E_Cj/E_L)**(1.0/4.0)*(a_phi+ad_phi))
phi=np.kron(Id_M,((E_Cj/E_L)**(1.0/4.0))*(a_phi+ad_phi))
print("kron PHI")
print(phi)
phi2=np.kron(Id_M,((E_Cj_2/E_L_2)**(1.0/4.0))*(a_phi+ad_phi))

#COS_THETA
costheta=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
costheta[0][2*M]+=1
costheta[2*M][0]+=1
print(costheta)
print("COS")
cos_theta=np.kron(costheta,Id_N)
print("kron COS")
print(cos_theta)

#N_PHI
N_phi_extended=np.kron(Id_M,N_phi)

#N_theta
n_theta_extended=np.kron(n_theta,Id_N)


#FOR SAVING VALUES OF CALCULATIONS

#1ST VALUES

ng1=np.arange(0.495,0.506,0.01)
print(ng1)
a=np.pi-(np.pi*0.01)
b=np.pi+(np.pi*0.01 + 0.001)
phi_ext1=np.arange(a,b,(0.01*np.pi)*2)
print(phi_ext1)
Energies=[]
Eigenvectors=[]

Expvals_costheta_NG=[]
Expvals_phi_NG=[]
Expvals_N_phi_NG=[]
Expvals_n_theta_NG=[]

Expvals_costheta_PHIEXT=[]
Expvals_phi_PHIEXT=[]
Expvals_N_phi_PHIEXT=[]
Expvals_n_theta_PHIEXT=[]

Overlaps_NG=[]
Overlaps_PHIEXT=[]

EIGENVECTS_NG=[]
EIGENVECTS_PHIEXT=[]

Overlaps_UP_NG=[]
Overlaps_UP_PHIEXT=[]
Overlaps_SIDE_NG=[]
Overlaps_SIDE_PHIEXT=[]
Overlaps_CROSSED_NG=[]
Overlaps_CROSSED_PHIEXT=[]

#2ND VALUES

ng2=np.arange(0.495,0.506,0.01)
phi_ext2=np.arange(a,b,(0.01*np.pi)*2)
Energies2=[]
Eigenvectors2=[]

Expvals2_costheta_NG=[]
Expvals2_phi_NG=[]
Expvals2_N_phi_NG=[]
Expvals2_n_theta_NG=[]

Expvals2_costheta_PHIEXT=[]
Expvals2_phi_PHIEXT=[]
Expvals2_N_phi_PHIEXT=[]
Expvals2_n_theta_PHIEXT=[]

Overlaps2_NG=[]
Overlaps2_PHIEXT=[]

EIGENVECTS2_NG=[]
EIGENVECTS2_PHIEXT=[]

Overlaps2_UP_NG=[]
Overlaps2_UP_PHIEXT=[]
Overlaps2_SIDE_NG=[]
Overlaps2_SIDE_PHIEXT=[]
Overlaps2_CROSSED_NG=[]
Overlaps2_CROSSED_PHIEXT=[]


PHI
[[0.         1.         0.         ... 0.         0.         0.        ]
 [1.         0.         1.41421356 ... 0.         0.         0.        ]
 [0.         1.41421356 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         7.         0.        ]
 [0.         0.         0.         ... 7.         0.         7.07106781]
 [0.         0.         0.         ... 0.         7.07106781 0.        ]]
kron PHI
[[0.         1.         0.         ... 0.         0.         0.        ]
 [1.         0.         1.41421356 ... 0.         0.         0.        ]
 [0.         1.41421356 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         7.         0.        ]
 [0.         0.         0.         ... 7.         0.         7.07106781]
 [0.         0.         0.         ... 0.         7.07106781 0.        ]]
[[0. 1. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1

In [5]:
# Construction of the 0-pi Hamiltonian

def Hamiltonian_0pi(EJ,ECS,ECJ,EL,n,phi,alph):
    #Initial
    H=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Free Hamiltonian construction
    H1=4*ECS*((n_theta-n*Id_M)@(n_theta-n*Id_M))
    H_theta=np.kron(H1,Id_N)
    H2=2*np.sqrt(ECJ*EL)*(N_phi+0.5*Id_N)
    H_phi=np.kron(Id_M,H2)
    H_0=H_theta+H_phi
    #H_0=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Interaction Hamiltonian construction
    cos_theta_ext=np.diag(np.ones(2*M),1)+np.diag(np.ones(2*M),-1) 
    #cos_theta=np.diag(np.ones(2*M-1),2)+np.diag(np.ones(2*M-1),-2)
    cos_theta_ext[0][2*M]+=1
    cos_theta_ext[2*M][0]+=1
    #cos_phi=(0.5*((displace(N+1,alph)*(math.e**(-1j*phi/2)))+(displace(N+1,-alph)*(math.e**(1j*phi/2)))))*np.diag(np.ones(N+1),0)
    cos_phi=0.5*(scipy.linalg.expm(alph*(a_phi+ad_phi)-(Id_N*1j*phi/2))+scipy.linalg.expm(-alph*(a_phi+ad_phi)+(Id_N*1j*phi/2)))
    H_I=EJ*np.kron(cos_theta_ext,cos_phi.real)#SIN EL2 
    #H_I=np.zeros(((2*M+1)*(N+1),(2*M+1)*(N+1)))
    #Total Hamiltonian
    H+=H_0-H_I
    return H

In [6]:
def calculate_expval(vect,operator):
    return np.conj(vect)@operator@vect

In [7]:
def calculate_overlap(vect1,vect2):
    return np.abs(np.conj(vect1)@vect2)**2

In [8]:
Vprueba=[1,1,0]
Vprueba2=[0,2,0]
print(Vprueba)
O=calculate_overlap(Vprueba,Vprueba2)
print("O=",O)
matrix=[[0,1,0],[1,0,1],[0,1,0]]
print(matrix)
a=calculate_expval(Vprueba,matrix)
print(a)


[1, 1, 0]
O= 4
[[0, 1, 0], [1, 0, 1], [0, 1, 0]]
2


In [9]:
Vs=[]
V1=np.array([1,1,0])
V2=np.array([0,2,0])
V3=np.array([1,1,1])
Vs.extend([V1,V2,V3])
print(Vs)
print(Vs[0])
print(calculate_overlap(Vs[0],Vs[1]))


[array([1, 1, 0]), array([0, 2, 0]), array([1, 1, 1])]
[1 1 0]
4


## $\text{1. Plot for } E_{Cs} = E_{Cj} = E_L = 1$, $\varphi_{ext}= \pi$, $\alpha = i \left(\frac{E_{CJ}}{E_L}\right)^{1/4} = i$.

In [10]:
# Calculations 1

for n1 in ng1:
    print("Step for ng = ",n1)
    H1=Hamiltonian_0pi(E_J,E_Cs,E_Cj,E_L,n1,phi_ext_val,alpha)
    print("Hamiltonian built, len(H)=",len(H1)**2)
    #vals,vects=np.linalg.eigh(H)
    valsNG, vectsNG = eigsh(H1, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",valsNG[0]," and ",valsNG[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vectsNG[:,0]," and ",vectsNG[:,1])
    EIGENVECTS_NG.extend([vectsNG[:,0],vectsNG[:,1]])
    print(EIGENVECTS_NG)
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval_costheta_0=calculate_expval(vectsNG[:,0],cos_theta)
    Expval_costheta_1=calculate_expval(vectsNG[:,1],cos_theta)
    #Phi
    Expval_phi_0=calculate_expval(vectsNG[:,0],phi)
    Expval_phi_1=calculate_expval(vectsNG[:,1],phi)
    #N_phi
    Expval_N_phi_0=calculate_expval(vectsNG[:,0],N_phi_extended)
    Expval_N_phi_1=calculate_expval(vectsNG[:,1],N_phi_extended)
    #n_theta
    Expval_n_theta_0=calculate_expval(vectsNG[:,0],n_theta_extended)
    Expval_n_theta_1=calculate_expval(vectsNG[:,1],n_theta_extended)
    #Overlap between the states of the same value of ng
    Overlap=calculate_overlap(vectsNG[:,0],vectsNG[:,1])
    print("Overlap = ",Overlap)
    #TOTALS
    Overlaps_NG.append(Overlap)
    Expvals_costheta_NG.extend([Expval_costheta_0,Expval_costheta_1])
    Expvals_phi_NG.extend([Expval_phi_0,Expval_phi_1])
    Expvals_N_phi_NG.extend([Expval_N_phi_0,Expval_N_phi_1])
    Expvals_n_theta_NG.extend([Expval_n_theta_0,Expval_n_theta_1])

Step for ng =  0.495
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.755031128287943  and  1.7889259142999934
The eigenvectors of the GS and the 1st excited state are  [-6.08784499e-18  7.07700471e-19  6.75295875e-18 ... -3.71673945e-18
 -4.53270859e-17  3.70989442e-17]  and  [-6.50943044e-18 -9.22163849e-19  1.59749310e-18 ... -1.30592203e-17
  1.75840730e-17 -3.65958664e-17]
[array([-6.08784499e-18,  7.07700471e-19,  6.75295875e-18, ...,
       -3.71673945e-18, -4.53270859e-17,  3.70989442e-17]), array([-6.50943044e-18, -9.22163849e-19,  1.59749310e-18, ...,
       -1.30592203e-17,  1.75840730e-17, -3.65958664e-17])]
Eigenvalues and eigenvectors calculated
Overlap =  5.887089309958741e-30
Step for ng =  0.505
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.755031128288037  and  1.78892591429994
The eigenvectors of the GS and the 1st excited state are  [-1.86261717e-18  3.34817903e-18  1.338104

In [11]:
# Calculations 1

for phi1 in phi_ext1:
    print("Step for phi_ext = ",phi1)
    H2=Hamiltonian_0pi(E_J,E_Cs,E_Cj,E_L,ng_val,phi1,alpha)
    print("Hamiltonian built, len(H)=",len(H2)**2)
    #vals,vects=np.linalg.eigh(H)
    valsPHI, vectsPHI = eigsh(H2, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",valsPHI[0]," and ",valsPHI[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vectsPHI[:,0]," and ",vectsPHI[:,1])
    EIGENVECTS_PHIEXT.extend([vectsPHI[:,0],vectsPHI[:,1]])
    print(EIGENVECTS_PHIEXT)
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval_costheta_0=calculate_expval(vectsPHI[:,0],cos_theta)
    Expval_costheta_1=calculate_expval(vectsPHI[:,1],cos_theta)
    #Phi
    Expval_phi_0=calculate_expval(vectsPHI[:,0],phi)
    Expval_phi_1=calculate_expval(vectsPHI[:,1],phi)
    #N_phi
    Expval_N_phi_0=calculate_expval(vectsPHI[:,0],N_phi_extended)
    Expval_N_phi_1=calculate_expval(vectsPHI[:,1],N_phi_extended)
    #n_theta
    Expval_n_theta_0=calculate_expval(vectsPHI[:,0],n_theta_extended)
    Expval_n_theta_1=calculate_expval(vectsPHI[:,1],n_theta_extended)
    #Overlap between the states of the same value of ng
    Overlap=calculate_overlap(vectsPHI[:,0],vectsPHI[:,1])
    print("Overlap = ",Overlap)
    #TOTALS
    Overlaps_PHIEXT.append(Overlap)
    Expvals_costheta_PHIEXT.extend([Expval_costheta_0,Expval_costheta_1])
    Expvals_phi_PHIEXT.extend([Expval_phi_0,Expval_phi_1])
    Expvals_N_phi_PHIEXT.extend([Expval_N_phi_0,Expval_N_phi_1])
    Expvals_n_theta_PHIEXT.extend([Expval_n_theta_0,Expval_n_theta_1])

Step for phi_ext =  3.1101767270538954
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.7631782525584474  and  1.7807346115216194
The eigenvectors of the GS and the 1st excited state are  [ 8.88391379e-18  1.37202166e-18  1.96090747e-19 ...  1.12794809e-17
 -1.21451125e-17 -8.05622842e-17]  and  [ 3.54625955e-17  8.16609251e-18  1.26261059e-18 ...  2.01773394e-17
 -2.30330006e-17 -6.07802080e-17]
[array([ 8.88391379e-18,  1.37202166e-18,  1.96090747e-19, ...,
        1.12794809e-17, -1.21451125e-17, -8.05622842e-17]), array([ 3.54625955e-17,  8.16609251e-18,  1.26261059e-18, ...,
        2.01773394e-17, -2.30330006e-17, -6.07802080e-17])]
Eigenvalues and eigenvectors calculated
Overlap =  7.703719777548943e-32
Step for phi_ext =  3.1730085801256913
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  1.7631782525585071  and  1.780734611521711
The eigenvectors of the GS and the 1st excited state are  [ 6

EXPECTATION VALUES FOR THE FIRST SET OF VALUES FOR THE ENERGIES

In [12]:
#VARYING NG
print("VARYING NG")
print("Exp values of cos_theta varying ng are ",Expvals_costheta_NG)
print("Exp values of phi varying ng are ",Expvals_phi_NG)
print("Exp values of N_phi varying ng are ",Expvals_N_phi_NG)
print("Exp values of n_theta varying ng are ",Expvals_n_theta_NG)
#VARYING PHI_EXT
print("VARYING PHI_EXT")
print("Exp values of cos_theta varying phi_ext are ",Expvals_costheta_PHIEXT)
print("Exp values of phi varying phi_ext are ",Expvals_phi_PHIEXT)
print("Exp values of N_phi varying phi_ext are ",Expvals_N_phi_PHIEXT)
print("Exp values of n_theta varying phi_ext are ",Expvals_n_theta_PHIEXT)

VARYING NG
Exp values of cos_theta varying ng are  [-4.904386026960489e-12, 4.90903686430856e-12, 2.5489710264758372e-12, -2.555202696491079e-12]
Exp values of phi varying ng are  [-2.58347985210531e-12, 2.6058027869727572e-12, 1.35277097978606e-12, -1.3295550816225492e-12]
Exp values of N_phi varying ng are  [0.0869314098219396, 0.09220055517257914, 0.08693140982191866, 0.09220055517257808]
Exp values of n_theta varying ng are  [0.07369822649556725, 0.9209886856432716, 0.9263017735044506, 0.07901131435671901]
VARYING PHI_EXT
Exp values of cos_theta varying phi_ext are  [1.0738891143108846, -1.0705380216291736, -1.0738891143109048, 1.0705380216291656]
Exp values of phi varying phi_ext are  [0.5556498947238198, -0.5619624882128206, -0.555649894723909, 0.5619624882127273]
Exp values of N_phi varying phi_ext are  [0.08873497723142418, 0.09026030865728792, 0.08873497723144974, 0.09026030865725901]
Exp values of n_theta varying phi_ext are  [0.4999999999974224, 0.5000000000025713, 0.5000000

In [13]:
Overlaps_UP_NG.extend([calculate_overlap(EIGENVECTS_NG[0],EIGENVECTS_NG[1]),calculate_overlap(EIGENVECTS_NG[2],EIGENVECTS_NG[3])])
Overlaps_UP_PHIEXT.extend([calculate_overlap(EIGENVECTS_PHIEXT[0],EIGENVECTS_PHIEXT[1]),calculate_overlap(EIGENVECTS_PHIEXT[2],EIGENVECTS_PHIEXT[3])])
Overlaps_SIDE_NG.extend([calculate_overlap(EIGENVECTS_NG[0],EIGENVECTS_NG[2]),calculate_overlap(EIGENVECTS_NG[1],EIGENVECTS_NG[3])])
Overlaps_SIDE_PHIEXT.extend([calculate_overlap(EIGENVECTS_PHIEXT[0],EIGENVECTS_PHIEXT[2]),calculate_overlap(EIGENVECTS_PHIEXT[1],EIGENVECTS_PHIEXT[3])])
Overlaps_CROSSED_NG.extend([calculate_overlap(EIGENVECTS_NG[0],EIGENVECTS_NG[3]),calculate_overlap(EIGENVECTS_NG[1],EIGENVECTS_NG[2])])
Overlaps_CROSSED_PHIEXT.extend([calculate_overlap(EIGENVECTS_PHIEXT[0],EIGENVECTS_PHIEXT[3]),calculate_overlap(EIGENVECTS_PHIEXT[1],EIGENVECTS_PHIEXT[2])])
print(Overlaps_UP_NG)
print(Overlaps_UP_PHIEXT)
print(Overlaps_SIDE_NG)
print(Overlaps_SIDE_PHIEXT)
print(Overlaps_CROSSED_NG)
print(Overlaps_CROSSED_PHIEXT)

[5.887089309958741e-30, 1.857099504183107e-30]
[7.703719777548943e-32, 2.0830858278492343e-30]
[1.1938519622032216e-24, 1.192229111230025e-24]
[1.3895206296112829e-24, 1.3853359197243117e-24]
[0.9999062295640823, 0.9999062295640738]
[0.9999750125015218, 0.9999750125015187]


## $\text{2. Plot for } E_{Cs} = \frac{1}{20}$, $E_{CJ} = 1$, $E_L = \frac{1}{16}$, $\varphi_{ext} = \pi$, $\alpha = i \left(\frac{E_{CJ}}{E_L}\right)^{1/4} = 2i$.

In [14]:
# Calculations 2

for n2 in ng2:
    print("Step for ng = ",n2)
    H3=Hamiltonian_0pi(E_J_2,E_Cs_2,E_Cj_2,E_L_2,n2,phi_ext_val,alpha2)
    print("Hamiltonian built, len(H)=",len(H3)**2)
    #vals2,vects2=np.linalg.eigh(H2)
    vals2NG, vects2NG = eigsh(H3, k=2, which='SA')
    #vals2,_=np.linalg.eigh(H3)
    print("The eigenvalues of the GS and the 1st excited state are ",vals2NG[0]," and ",vals2NG[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vects2NG[:,0]," and ",vects2NG[:,1])
    EIGENVECTS2_NG.extend([vects2NG[:,0],vects2NG[:,1]])
    print(EIGENVECTS2_NG)
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval2_costheta_0=calculate_expval(vects2NG[:,0],cos_theta)
    Expval2_costheta_1=calculate_expval(vects2NG[:,1],cos_theta)
    #Phi
    Expval2_phi_0=calculate_expval(vects2NG[:,0],phi)
    Expval2_phi_1=calculate_expval(vects2NG[:,1],phi)
    #N_phi
    Expval2_N_phi_0=calculate_expval(vects2NG[:,0],N_phi_extended)
    Expval2_N_phi_1=calculate_expval(vects2NG[:,1],N_phi_extended)
    #n_theta
    Expval2_n_theta_0=calculate_expval(vects2NG[:,0],n_theta_extended)
    Expval2_n_theta_1=calculate_expval(vects2NG[:,1],n_theta_extended)
    #Overlap between the states of the same value of ng
    Overlap2=calculate_overlap(vects2NG[:,0],vects2NG[:,1])
    print("Overlap = ",Overlap2)
    #TOTALS
    Overlaps2_NG.append(Overlap2)
    Expvals2_costheta_NG.extend([Expval2_costheta_0,Expval2_costheta_1])
    Expvals2_phi_NG.extend([Expval2_phi_0,Expval2_phi_1])
    Expvals2_N_phi_NG.extend([Expval2_N_phi_0,Expval2_N_phi_1])
    Expvals2_n_theta_NG.extend([Expval2_n_theta_0,Expval2_n_theta_1])

Step for ng =  0.495
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2741644560315866  and  0.2761153906221494
The eigenvectors of the GS and the 1st excited state are  [ 1.64057624e-18 -4.88529043e-18  3.37342603e-19 ...  1.12442776e-17
  7.26843034e-17  1.51020147e-16]  and  [-7.14089941e-19  5.09359610e-18  3.63877379e-19 ...  9.81678750e-18
 -2.26242821e-16 -2.72252328e-16]
[array([ 1.64057624e-18, -4.88529043e-18,  3.37342603e-19, ...,
        1.12442776e-17,  7.26843034e-17,  1.51020147e-16]), array([-7.14089941e-19,  5.09359610e-18,  3.63877379e-19, ...,
        9.81678750e-18, -2.26242821e-16, -2.72252328e-16])]
Eigenvalues and eigenvectors calculated
Overlap =  4.1237259547523365e-31
Step for ng =  0.505
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2741644560315752  and  0.27611539062212354
The eigenvectors of the GS and the 1st excited state are  [-2.75857396e-18 -5.23696132e-18 -9.

In [15]:
# Calculations 2

for phi2 in phi_ext2:
    print("Step for phi_ext = ",phi2)
    H4=Hamiltonian_0pi(E_J_2,E_Cs_2,E_Cj_2,E_L_2,ng_val,phi2,alpha2)
    print("Hamiltonian built, len(H)=",len(H4)**2)
    #vals,vects=np.linalg.eigh(H)
    vals2PHI, vects2PHI = eigsh(H4, k=2, which='SA')
    #vals,_=np.linalg.eigh(H)
    print("The eigenvalues of the GS and the 1st excited state are ",vals2PHI[0]," and ",vals2PHI[1])
    print("The eigenvectors of the GS and the 1st excited state are ",vects2PHI[:,0]," and ",vects2PHI[:,1])
    EIGENVECTS2_PHIEXT.extend([vects2PHI[:,0],vects2PHI[:,1]])
    print(EIGENVECTS2_PHIEXT)
    print("Eigenvalues and eigenvectors calculated")
    #Cos_theta
    Expval2_costheta_0=calculate_expval(vects2PHI[:,0],cos_theta)
    Expval2_costheta_1=calculate_expval(vects2PHI[:,1],cos_theta)
    #Phi
    Expval2_phi_0=calculate_expval(vects2PHI[:,0],phi)
    Expval2_phi_1=calculate_expval(vects2PHI[:,1],phi)
    #N_phi
    Expval2_N_phi_0=calculate_expval(vects2PHI[:,0],N_phi_extended)
    Expval2_N_phi_1=calculate_expval(vects2PHI[:,1],N_phi_extended)
    #n_theta
    Expval2_n_theta_0=calculate_expval(vects2PHI[:,0],n_theta_extended)
    Expval2_n_theta_1=calculate_expval(vects2PHI[:,1],n_theta_extended)
    #Overlap between the states of the same value of ng
    Overlap2=calculate_overlap(vects2PHI[:,0],vects2PHI[:,1])
    print("Overlap = ",Overlap2)
    #TOTALS
    Overlaps2_PHIEXT.append(Overlap2)
    Expvals2_costheta_PHIEXT.extend([Expval2_costheta_0,Expval2_costheta_1])
    Expvals2_phi_PHIEXT.extend([Expval2_phi_0,Expval2_phi_1])
    Expvals2_N_phi_PHIEXT.extend([Expval2_N_phi_0,Expval2_N_phi_1])
    Expvals2_n_theta_PHIEXT.extend([Expval2_n_theta_0,Expval2_n_theta_1])

Step for phi_ext =  3.1101767270538954
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.27470737232471343  and  0.275564077004385
The eigenvectors of the GS and the 1st excited state are  [-8.59162942e-18  3.87076612e-19 -7.32967952e-18 ...  3.91319529e-17
 -5.81048955e-18 -1.03358549e-16]  and  [ 1.08267710e-17  3.32852904e-18  1.11943439e-17 ...  3.00383302e-18
 -3.44303255e-17  6.35957422e-17]
[array([-8.59162942e-18,  3.87076612e-19, -7.32967952e-18, ...,
        3.91319529e-17, -5.81048955e-18, -1.03358549e-16]), array([ 1.08267710e-17,  3.32852904e-18,  1.11943439e-17, ...,
        3.00383302e-18, -3.44303255e-17,  6.35957422e-17])]
Eigenvalues and eigenvectors calculated
Overlap =  9.663546088957395e-30
Step for phi_ext =  3.1730085801256913
Hamiltonian built, len(H)= 4372281
The eigenvalues of the GS and the 1st excited state are  0.2747073723247273  and  0.2755640770043964
The eigenvectors of the GS and the 1st excited state are  [-

EXPECTATION VALUES FOR THE SECOND SET OF VALUES FOR THE ENERGIES

In [16]:
#VARYING NG
print("VARYING NG")
print("Exp values of cos_theta varying ng are ",Expvals2_costheta_NG)
print("Exp values of phi varying ng are ",Expvals2_phi_NG)
print("Exp values of N_phi varying ng are ",Expvals2_N_phi_NG)
print("Exp values of n_theta varying ng are ",Expvals2_n_theta_NG)
#VARYING PHI_EXT
print("VARYING PHI_EXT")
print("Exp values of cos_theta varying phi_ext are ",Expvals2_costheta_PHIEXT)
print("Exp values of phi varying phi_ext are ",Expvals2_phi_PHIEXT)
print("Exp values of N_phi varying phi_ext are ",Expvals2_N_phi_PHIEXT)
print("Exp values of n_theta varying phi_ext are ",Expvals2_n_theta_PHIEXT)

VARYING NG
Exp values of cos_theta varying ng are  [-6.7757418785871574e-12, 6.7640561098692655e-12, 9.019136055111067e-12, -9.025094886879884e-12]
Exp values of phi varying ng are  [-1.3576321106753455e-12, 1.3442505422808337e-12, 1.8087554022072953e-12, -1.8239964414875957e-12]
Exp values of N_phi varying ng are  [0.04402386741254899, 0.04424840052522609, 0.04402386741254689, 0.04424840052522459]
Exp values of n_theta varying ng are  [0.012071718350909273, 0.9875368300487428, 0.9879282816490927, 0.012463169951257633]
VARYING PHI_EXT
Exp values of cos_theta varying phi_ext are  [1.0837543531326714, -1.0800974073986416, -1.0837543531326737, 1.0800974073986171]
Exp values of phi varying phi_ext are  [0.21752466351105393, -0.21875833128052935, -0.217524663511057, 0.21875833128047284]
Exp values of N_phi varying phi_ext are  [0.04409616925676945, 0.04417101278913896, 0.044096169256768014, 0.044171012789131395]
Exp values of n_theta varying phi_ext are  [0.5000000000096448, 0.4999999999903

In [17]:
Overlaps2_UP_NG.extend([calculate_overlap(EIGENVECTS2_NG[0],EIGENVECTS2_NG[1]),calculate_overlap(EIGENVECTS2_NG[2],EIGENVECTS2_NG[3])])
Overlaps2_UP_PHIEXT.extend([calculate_overlap(EIGENVECTS2_PHIEXT[0],EIGENVECTS2_PHIEXT[1]),calculate_overlap(EIGENVECTS2_PHIEXT[2],EIGENVECTS2_PHIEXT[3])])
Overlaps2_SIDE_NG.extend([calculate_overlap(EIGENVECTS2_NG[0],EIGENVECTS2_NG[2]),calculate_overlap(EIGENVECTS2_NG[1],EIGENVECTS2_NG[3])])
Overlaps2_SIDE_PHIEXT.extend([calculate_overlap(EIGENVECTS2_PHIEXT[0],EIGENVECTS2_PHIEXT[2]),calculate_overlap(EIGENVECTS2_PHIEXT[1],EIGENVECTS2_PHIEXT[3])])
Overlaps2_CROSSED_NG.extend([calculate_overlap(EIGENVECTS2_NG[0],EIGENVECTS2_NG[3]),calculate_overlap(EIGENVECTS2_NG[1],EIGENVECTS2_NG[2])])
Overlaps2_CROSSED_PHIEXT.extend([calculate_overlap(EIGENVECTS2_PHIEXT[0],EIGENVECTS2_PHIEXT[3]),calculate_overlap(EIGENVECTS2_PHIEXT[1],EIGENVECTS2_PHIEXT[2])])
print(Overlaps2_UP_NG)
print(Overlaps2_UP_PHIEXT)
print(Overlaps2_SIDE_NG)
print(Overlaps2_SIDE_PHIEXT)
print(Overlaps2_CROSSED_NG)
print(Overlaps2_CROSSED_PHIEXT)

[4.1237259547523365e-31, 1.6446232270027758e-31]
[9.663546088957395e-30, 1.925929944387236e-30]
[1.106749916292934e-24, 1.106220076275703e-24]
[6.377532135122217e-23, 6.380369628355252e-23]
[0.9999983013475612, 0.9999983013475607]
[0.9999833013646454, 0.9999833013646436]


In [18]:
EIGVECTS_NG = np.array(EIGENVECTS_NG)
np.save('EIGVECTS_NG.npy', EIGVECTS_NG)
EIGVECTS_PHIEXT = np.array(EIGENVECTS_PHIEXT)
np.save('EIGVECTS_PHIEXT.npy', EIGVECTS_PHIEXT)
EIGVECTS2_NG = np.array(EIGENVECTS2_NG)
np.save('EIGVECTS2_NG.npy', EIGVECTS2_NG)
EIGVECTS2_PHIEXT = np.array(EIGENVECTS2_PHIEXT)
np.save('EIGVECTS2_PHIEXT.npy', EIGVECTS2_PHIEXT)